In [2]:
from __future__ import print_function
import numpy as np
import pandas as pd
import pickle
from collections import defaultdict
import re
from bs4 import BeautifulSoup
import sys
import os

os.environ['KERAS_BACKEND'] = 'theano'

from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical

from keras.layers import Embedding
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding, Dropout, Add
from keras.models import Model

MAX_SEQUENCE_LENGTH = 1000
MAX_NB_WORDS = 20000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2
DATA_DIR = '/content/drive/MyDrive/Colab Notebooks/диплом'

def clean_str(string):
    """
    Tokenization/string cleaning for dataset
    Every dataset is lower cased except
    """
    string = re.sub(r"\\", "", string)    
    string = re.sub(r"\'", "", string)    
    string = re.sub(r"\"", "", string)    
    return string.strip().lower()

data_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/диплом/export.csv', sep=',')
print(data_train.shape)

(2716, 7)


In [3]:
data_train.head()

,N,Тип,Источник,Содержание сообщения,Оригинал сообщения,Дата,Unnamed: 6
0,1,Комментарий к посту/репосту,Группа,"Действительно,кто эту провозгласил примадонной...",0,11.05.2022 11:33:25,NaN
1,2,Комментарий к посту/репосту,Группа,"Схем много,спасибо,но по всем связать пенсии н...",0,10.05.2022 19:13:27,NaN
2,3,Комментарий к посту/репосту,Пользователь,"И найдутся же дураки,кто это чмо купит.",0,10.05.2022 19:01:50,NaN
3,4,Комментарий к посту/репосту,Группа,"Из бисера делаю эти подснежники,а хотелось бы ...",0,10.05.2022 18:53:50,NaN
4,5,Комментарий к посту/репосту,Группа,"Да пристрелить уже этого педика ,из ума выжил ...",0,10.05.2022 18:21:55,NaN


In [4]:
data = pd.DataFrame()
data['content'] = data_train['Содержание сообщения']
data['label'] = data_train['Оригинал сообщения']
data.dropna() 
data.head()

,content,label
0,"Действительно,кто эту провозгласил примадонной...",0
1,"Схем много,спасибо,но по всем связать пенсии н...",0
2,"И найдутся же дураки,кто это чмо купит.",0
3,"Из бисера делаю эти подснежники,а хотелось бы ...",0
4,"Да пристрелить уже этого педика ,из ума выжил ...",0


In [5]:
texts = []
labels = []

for idx in range(data.content.shape[0]):
    try:
        if len(data.label[idx]) == 1:
            text = BeautifulSoup(data.content[idx])
            # print(data.label[idx])
            texts.append(clean_str(text.get_text()))
            labels.append(data.label[idx])
    except:
        print(idx)
    

tokenizer = Tokenizer(nb_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

379
2626
2680
2681
2682
2683
2684
2685
2686
2687
2688
2689
2690
2691
2692
2693
2694
2705


/usr/local/lib/python3.8/dist-packages/keras/preprocessing/text.py:246: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn(


Found 13739 unique tokens.


In [ ]:
len(texts)

2694

In [6]:
data_pad = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
labels = np.asarray(labels)
# labels = to_categorical(np.asarray(labels))
print(('Shape of data tensor:', data_pad.shape))
print(('Shape of label tensor:', labels.shape))

indices = np.arange(len(data_pad))
np.random.shuffle(indices)
data_pad = data_pad[indices]
labels = labels[indices]
nb_validation_samples = int(VALIDATION_SPLIT * data_pad.shape[0])
X_train = texts[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
X_test = texts[-nb_validation_samples:]
y_test = labels[-nb_validation_samples:]
# print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
print('Number of positive and negative reviews in traing and validation set ')
# print(y_train.sum(axis=0))
# print(y_test.sum(axis=0))

('Shape of data tensor:', (2694, 1000))
('Shape of label tensor:', (2694,))
Number of positive and negative reviews in traing and validation set 


In [7]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', RandomForestClassifier(n_estimators=100)),
                     ])

text_clf.fit(X_train, y_train)


predicted = text_clf.predict(X_test)

print(metrics.classification_report(y_test, predicted))

              precision    recall  f1-score   support

           0       0.91      1.00      0.95       490
           1       0.00      0.00      0.00         5
           2       0.00      0.00      0.00        37
           3       0.00      0.00      0.00         3
           4       0.00      0.00      0.00         1
           5       0.00      0.00      0.00         2

    accuracy                           0.91       538
   macro avg       0.15      0.17      0.16       538
weighted avg       0.83      0.91      0.87       538



/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
